In [ ]:
# import time
# from pybit import spot
# import math

# # Configura la tua chiave API Bybit e il segreto
# api_key = 'YOUR_API_KEY'
# api_secret = 'YOUR_API_SECRET'

# # Inizializza la connessione con Bybit Spot
# global_client = spot.HTTP(api_key=api_key, api_secret=api_secret)

# # Variabili iniziali
# pair = "BTCUSDT"
# liquidita_totale = None
# bitcoin_totale = None
# prezzo_medio_pesato = 0
# ultima_liquidita_usata = 0

# # Ottieni la liquidita totale iniziale
# def aggiorna_liquidita_e_bitcoin():
#     global liquidita_totale, bitcoin_totale
#     balance = global_client.get_wallet_balance()["balances"]
#     for asset in balance:
#         if asset['coin'] == 'USDT':
#             liquidita_totale = float(asset['free'])
#         elif asset['coin'] == 'BTC':
#             bitcoin_totale = float(asset['free'])

# # Controlla se due valori sono molto simili, usando la funzione math.isclose
# def valori_simili(val1, val2):
#     return math.isclose(val1, val2, rel_tol=1e-5)

# # Calcola il prezzo medio pesato
# def aggiorna_prezzo_medio_pesato(prezzo_acquisto, liquidita_usata):
#     global prezzo_medio_pesato, ultima_liquidita_usata
#     ultima_liquidita_usata = liquidita_usata
#     if liquidita_totale != 0:
#         prezzo_medio_pesato = ((prezzo_medio_pesato * (liquidita_totale - liquidita_usata)) + (prezzo_acquisto * liquidita_usata)) / liquidita_totale


# def trading():
#     global prezzo_medio_pesato
#     aggiorna_liquidita_e_bitcoin()
    
#     while True:
#         # Compra BTC a mercato usando l'1% della liquidita rimasta
#         liquidita_da_usare = 0.01 * liquidita_totale
#         if liquidita_da_usare < 10:  # Bybit ha un ordine minimo per il mercato spot
#             print("Liquidita insufficiente per effettuare un ordine.")
#             break
        
#         order = global_client.place_active_order(
#             symbol=pair,
#             side="Buy",
#             type="MARKET",
#             qty=liquidita_da_usare / global_client.latest_price(symbol=pair)['price']
#         )
#         prezzo_acquisto = float(order['price'])
#         aggiorna_prezzo_medio_pesato(prezzo_acquisto, liquidita_da_usare)
#         aggiorna_liquidita_e_bitcoin()
        
#         # Controllo per vedere se la liquidita e il balance sono simili
#         assert valori_simili(liquidita_totale, float(order['origQty']) * float(order['price'])), "Discrepanza tra liquidita in Python e su Bybit!"
        
#         # Attendi 90 secondi
#         time.sleep(90)
        
#         # Guarda il prezzo attuale
#         prezzo_corrente = float(global_client.latest_price(symbol=pair)['price'])
        
#         # Se il prezzo corrente è inferiore a (prezzo medio pesato degli acquisti fatti / 0.999), compra ancora
#         if prezzo_corrente < prezzo_medio_pesato / 0.999:
#             aggiorna_liquidita_e_bitcoin()
#             liquidita_da_usare = 0.01 * liquidita_totale + ultima_liquidita_usata
#             if liquidita_da_usare < 10:  # Bybit ha un ordine minimo per il mercato spot
#                 print("Liquidita insufficiente per effettuare un ordine.")
#                 break
            
#             order = global_client.place_active_order(
#                 symbol=pair,
#                 side="Buy",
#                 type="MARKET",
#                 qty=liquidita_da_usare / global_client.latest_price(symbol=pair)['price']
#             )
#             prezzo_acquisto = float(order['price'])
#             aggiorna_prezzo_medio_pesato(prezzo_acquisto, liquidita_da_usare)
#             aggiorna_liquidita_e_bitcoin()
#         # Se il prezzo corrente è superiore a (prezzo medio pesato degli acquisti fatti / 0.999), vendi tutto
#         elif prezzo_corrente > prezzo_medio_pesato / 0.999:
#             aggiorna_liquidita_e_bitcoin()
#             if bitcoin_totale * prezzo_corrente < 10:  # Bybit ha un ordine minimo per il mercato spot
#                 print("Quantita di Bitcoin insufficiente per effettuare un ordine di vendita.")
#                 break
            
#             order = global_client.place_active_order(
#                 symbol=pair,
#                 side="Sell",
#                 type="MARKET",
#                 qty=bitcoin_totale
#             )
#             prezzo_vendita = float(order['price'])
#             prezzo_medio_pesato = 0  # Resetta il prezzo medio pesato
#             aggiorna_liquidita_e_bitcoin()
        
#         # Controllo per vedere se la liquidita e il balance sono simili
#         assert valori_simili(bitcoin_totale, float(order['origQty'])), "Discrepanza tra bitcoin in Python e su Bybit!"


# if __name__ == "__main__":
#     trading()

In [1]:
import ccxt
import config_API_Bybit as cab

# Inizializza l'exchange Bybit
exchange = ccxt.bybit({
    'apiKey': cab.api_key,
    'secret': cab.api_secret,
    'enableRateLimit': True,
    'options': {
        'adjustForTimeDifference': True,
    },
})

# exchange.options['recvWindow'] = 10000  # Imposta a 10 secondi
# exchange.verbose = True

# Se desideri utilizzare il testnet
exchange.set_sandbox_mode(False)
# Carica i mercati
exchange.load_markets()

print('Simboli:\n', exchange.symbols)

# Ottieni il saldo del portafoglio
balance = exchange.fetch_balance()
balance

Simboli:
 ['10000000AIDOGE/USDT:USDT', '1000000BABYDOGE/USDT:USDT', '1000000CHEEMS/USDT:USDT', '1000000MOG/USDT:USDT', '1000000PEIPEI/USDT:USDT', '10000COQ/USDT:USDT', '10000LADYS/USDT:USDT', '10000SATS/USDT:USDT', '10000WEN/USDT:USDT', '10000WHY/USDT:USDT', '1000APU/USDT:USDT', '1000BONK/USDC:USDC', '1000BONK/USDT:USDT', '1000BTT/USDT:USDT', '1000CAT/USDT:USDT', '1000CATS/USDT:USDT', '1000FLOKI/USDT:USDT', '1000LUNC/USDT:USDT', '1000MUMU/USDT:USDT', '1000NEIROCTO/USDT:USDT', '1000PEPE/USDC:USDC', '1000PEPE/USDT:USDT', '1000RATS/USDT:USDT', '1000TURBO/USDT:USDT', '1000X/USDT:USDT', '1000XEC/USDT:USDT', '1CAT/USDT:USDT', '1INCH/USDT', '1INCH/USDT:USDT', '1SOL/USDT', '3P/USDT', '5IRE/USDT', 'A8/USDT', 'A8/USDT:USDT', 'AARK/USDT', 'AAVE/USDT', 'AAVE/USDT:USDT', 'ACA/USDT', 'ACE/USDT:USDT', 'ACH/USDT', 'ACH/USDT:USDT', 'ACS/USDT', 'ACT/USDT:USDT', 'ADA/EUR', 'ADA/USD:ADA', 'ADA/USDC', 'ADA/USDT', 'ADA/USDT:USDT', 'AEG/USDT', 'AERGO/USDT:USDT', 'AERO/USDT:USDT', 'AEVO/USDC:USDC', 'AEVO/USDT

{'info': {'retCode': '0',
  'retMsg': 'OK',
  'result': {'list': [{'totalEquity': '1607.64778747',
     'accountIMRate': '',
     'totalMarginBalance': '',
     'totalInitialMargin': '',
     'accountType': 'UNIFIED',
     'totalAvailableBalance': '',
     'accountMMRate': '',
     'totalPerpUPL': '0',
     'totalWalletBalance': '1607.64778747',
     'accountLTV': '',
     'totalMaintenanceMargin': '',
     'coin': [{'availableToBorrow': '',
       'bonus': '0',
       'accruedInterest': '0',
       'availableToWithdraw': '0.00004879',
       'totalOrderIM': '0',
       'equity': '0.00004879',
       'totalPositionMM': '0',
       'usdValue': '4.6836855',
       'unrealisedPnl': '0',
       'collateralSwitch': False,
       'spotHedgingQty': '0',
       'borrowAmount': '0',
       'totalPositionIM': '0',
       'walletBalance': '0.00004879',
       'cumRealisedPnl': '-0.00000021',
       'locked': '0',
       'marginCollateral': True,
       'coin': 'BTC'},
      {'availableToBorrow': 

In [11]:
import ccxt
import config_API_Bybit as cab

# Inizializza l'exchange Bybit
exchange = ccxt.bybit({
    'apiKey': cab.api_key,
    'secret': cab.api_secret,
    'enableRateLimit': True,
    'options': {
        'adjustForTimeDifference': True,
    },
})

# Disabilita la modalità sandbox per l'ambiente live
exchange.set_sandbox_mode(False)

# Carica i mercati
exchange.load_markets()

def get_balance(crypto_symbol, account_type):
    """
    Recupera il saldo di una specifica criptovaluta in un determinato account.

    :param crypto_symbol: Simbolo della criptovaluta (es. 'FOXY')
    :param account_type: Tipo di account (es. 'spot', 'funding', 'unified')
    :return: Dizionario con i saldi 'free', 'used' e 'total'
    """
    try:
        # Recupera il saldo per il tipo di account specificato
        balance = exchange.fetch_balance({'type': account_type})
        # Ottieni il saldo della criptovaluta specificata
        crypto_balance = balance.get(crypto_symbol, {'free': 0, 'used': 0, 'total': 0})
        return crypto_balance
    except Exception as e:
        print(f"Errore durante il recupero del saldo per {crypto_symbol} nell'account {account_type}: {e}")
        return None

# Esempio di utilizzo
crypto = 'USDT'
account_types = ['spot', 'funding', 'unified']

for account in account_types:
    balance = get_balance(crypto, account)
    if balance:
        print(f"Saldo di {crypto} nell'account {account}:")
        print(f"  Disponibile: {balance['free']}")
        print(f"  In uso: {balance['used']}")
        print(f"  Totale: {balance['total']}\n")

InvalidNonce: bybit {"retCode":10002,"retMsg":"invalid request, please check your server timestamp or recv_window param. req_timestamp[1733433560849],server_timestamp[1733433559431],recv_window[5000]","result":{},"retExtInfo":{},"time":1733433559431}